In [42]:
import nltk
from collections import Counter
import glob
import os
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Paths for all the different corpora
folderpaths = ['../Corpora/AnxietyMeditations/', '../Corpora/SleepMeditations/',
               '../Corpora/LearningAndGrowthMeditations/', '../Corpora/MorningMeditations/']

#array to store the text of each file 
corpus = []
text2 = ""

for folder in folderpaths:
    if folder == '../Corpora/AnxietyMeditations/':
        for doc in glob.glob(os.path.join(folder, '*.txt')):
            with open(doc, 'r') as f:
                text = f.read()
                corpus.append(text)
                text2 += text
    elif folder == '../Corpora/SleepMeditations/':
        for doc in glob.glob(os.path.join(folder, '*.txt')):
            with open(doc, 'r') as f:
                text = f.read()
                corpus.append(text)
                text2 += text
    elif folder == '../Corpora/LearningAndGrowthMeditations/':
        for doc in glob.glob(os.path.join(folder, '*.txt')):
            with open(doc, 'r') as f:
                text = f.read()
                corpus.append(text)
                text2 += text
    elif folder == '../Corpora/MorningMeditations/':
        for doc in glob.glob(os.path.join(folder, '*.txt')):
            with open(doc, 'r') as f:
                text = f.read()
                corpus.append(text)
                text2 += text

newStopWords = ['[', ']', '`', '#', '$',
                '(', ')', "Music", ':', "....", "--", ",", "''", '`', '"']
stopwords = set(stopwords.words('english') +
                newStopWords + list(string.punctuation))

# Tokenizing by word and filtering stopwords
all_tokens = [w for w in word_tokenize(text2.lower()) if not w in stopwords]

In [43]:
import nltk
from nltk.tokenize import word_tokenize

# Preprocess the text data
tokenized_corpus = []
for text in corpus:
    # Tokenize and lowercase
    tokens = [w for w in word_tokenize(text.lower()) if not w in stopwords]
    tokenized_corpus.append(tokens)


In [44]:
import os
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
# Preprocess the text data
tokenized_corpus = []
for text in corpus:
    tokens = word_tokenize(text.lower())  # Tokenize and lowercase
    tokenized_corpus.append(tokens)

context_windows = []
window_size = 4  # Size of the context window

for sentence in tokenized_corpus:
    sentence_length = len(sentence)
    for i in range(sentence_length):
        # Extract the target word
        target_word = sentence[i]

        # Extract the context words within the window size
        context_words = []
        for j in range(max(0, i - window_size), min(sentence_length, i + window_size + 1)):
            if j != i:
                context_words.append(sentence[j])

        # Append the target word and its context words as a tuple
        # Convert context_words to tuple
        context_windows.append((target_word, tuple(context_words)))

# Train the Word2Vec model
model = Word2Vec(context_windows, window=5,
                 min_count=10, sg=0)  # CBOW approach

# Accessing trained word vectors
word_vector = model.wv['breathe']  # Get the word vector for a specific word
print(word_vector)

[-0.02571531  0.06281108  0.02818606  0.11388033  0.06083102 -0.04686197
  0.09241629  0.03582029 -0.08555122 -0.00934544 -0.00797304 -0.11676246
  0.01230918 -0.06387286  0.00378325  0.02980082 -0.01165971 -0.08301544
 -0.02056445 -0.21911184 -0.00169653  0.08718224  0.08829284 -0.06478397
 -0.02349795  0.08676254 -0.04553212 -0.02165003 -0.12373185  0.01578454
  0.079062    0.03106468  0.05743916 -0.01540777 -0.10006491  0.08969834
  0.00525573 -0.00038158 -0.05550962 -0.12587121  0.09258448 -0.03257527
 -0.01509936 -0.05683074  0.09033899 -0.03751928 -0.07906368  0.06086323
  0.0907647   0.01035591 -0.01625794 -0.0197103   0.01278726  0.00644125
 -0.0809281   0.06339431  0.00782283 -0.07654206 -0.02362784 -0.02029045
  0.020501    0.04719731  0.07319804  0.02503927 -0.02815674  0.03559386
 -0.12134167  0.02501273 -0.09099378 -0.03016839 -0.01211414  0.08106922
  0.01969916 -0.02452764  0.03276649 -0.03212637  0.03407028 -0.09734505
 -0.04621753 -0.06529494 -0.00386717 -0.05495413 -0

In [45]:
import os
import nltk
from gensim.models import Word2Vec

# Define the directory containing the text files
directory = "./txt Files/"

all_vectors = { 'sight': [], 'smell': [], 'sound': [], 'taste': []
, 'touch': []}

# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        filepath = os.path.join(directory, filename)
        with open(filepath, "r") as f:
            text = f.read()
            words = nltk.word_tokenize(text)
            for word in words:
                try:
                    word_vector = model.wv[word.lower()]
                    all_vectors[filename[:-4]].append(word_vector)
                    # print(f"Word: {word}\nVector: {word_vector}\n")
                except KeyError:
                    continue
                    # print(f"Word '{word}' not in vocabulary.\n")

# print(len(all_vectors['sight']))
# print(len(all_vectors['sight']))


In [46]:
import numpy as np
from scipy.spatial.distance import cosine

for key in all_vectors.keys(): 
    # Calculate the pairwise cosine similarity distance between each row in the matrix
    sight_len = len(all_vectors[key])
    distances = np.zeros((sight_len, sight_len))
    for i in range(sight_len):
        for j in range(i+1, sight_len):
            distances[i][j] = cosine(all_vectors[key][i], all_vectors[key][j])
            distances[j][i] = distances[i][j]

    # Print the resulting distance matrix
    distances = distances[:15, :15]
    print(distances.shape)

# Normalize the matrix to the range 0-1
# normalized_distance = (distances - np.min(distances)) / (np.max(distances) - np.min(distances))

(15, 15)
(15, 15)
(15, 15)
(15, 15)
(15, 15)
